## Introduction

I am using my home PC, with Debian 12 "bookworm" (testing) and an NVIDIA GPU.

This document is mostly for my own reference. It wasn't so easy getting everything working nicely and I want to have a record of how I did it!

Initially I wanted to avoid using docker, conda, or virtualenv. Instead, I installed the necessary Python modules under `/usr/local` with `pip`. I was thinking that I would be able to use my AI tools directly from the command line and other scripts, like any other tools.

This worked for a little while, but then Debian started using Python 3.11 for their default Python. Torch isn't compatible with Python 3.11 yet, and everything broke for me. After trying to fix it for a while and encountering numerous problems, I decided to switch to using venvs.

## Debian apt setup

I installed CUDA from NVIDIA's repositories which are intended for Ubuntu, which is a bit hacky, but it works.

### Don't break Debian, release pinning

In [ ]:
cat /etc/apt/preferences.d/99dontbreakdebian

Package: *
Pin: release o=Debian,a=experimental
Pin-Priority: 1

Package: *
Pin: release o=Debian,a=unstable
Pin-Priority: 90

Package: *
Pin: release a=focal
Pin-Priority: 70

Package: *
Pin: release a=jammy
Pin-Priority: 80

Package: *
Pin: release o=LP-PPA-deadsnakes
Pin-Priority: 90

Package: *
Pin: origin ppa.launchpad.net
Pin-Priority: 90

Package: *
Pin: release o=Debian
Pin-Priority: 990


### My main apt sources.list

In [ ]:
cat /etc/apt/sources.list

deb http://deb.debian.org/debian/ bookworm main contrib non-free
deb http://deb.debian.org/debian/ sid main contrib non-free
deb-src http://deb.debian.org/debian/ bookworm main contrib non-free
deb-src http://deb.debian.org/debian/ sid main contrib non-free

deb http://security.debian.org/debian-security bookworm-security main contrib non-free
deb-src http://security.debian.org/debian-security bookworm-security main contrib non-free

deb http://deb.debian.org/debian/ bookworm-updates main contrib non-free
deb-src http://deb.debian.org/debian/ bookworm-updates main contrib non-free

deb http://deb.debian.org/debian/ bookworm-backports main contrib non-free
deb-src http://deb.debian.org/debian/ bookworm-backports main contrib non-free

deb http://deb.debian.org/debian/ experimental main contrib non-free
deb-src http://deb.debian.org/debian/ experimental main contrib non-free


### Apt sources for NVIDIA CUDA, CUDNN, TensorRT, and containers

In [ ]:
cd /etc/apt/sources.list.d

In [ ]:
cat cuda-debian11-x86_64.list

deb [signed-by=/usr/share/keyrings/cuda-archive-keyring.gpg] https://developer.download.nvidia.com/compute/cuda/repos/debian11/x86_64/ /


In [ ]:
cat cudnn-local-ubuntu2204-8.6.0.163.list

deb [signed-by=/usr/share/keyrings/cudnn-local-FAED14DD-keyring.gpg] file:///var/cudnn-local-repo-ubuntu2204-8.6.0.163 /


In [ ]:
cat nv-tensorrt-local-ubuntu2204-8.5.3-cuda-11.8.list

deb [signed-by=/usr/share/keyrings/nv-tensorrt-local-3E951519-keyring.gpg] file:///var/nv-tensorrt-local-repo-ubuntu2204-8.5.3-cuda-11.8 /


In [ ]:
cat nvidia-container-runtime.list

deb https://nvidia.github.io/libnvidia-container/stable/debian11/$(ARCH) /
# deb https://nvidia.github.io/libnvidia-container/experimental/debian10/$(ARCH) /
deb https://nvidia.github.io/nvidia-container-runtime/stable/debian11/$(ARCH) /
# deb https://nvidia.github.io/nvidia-container-runtime/experimental/debian10/$(ARCH) /


### Apt sources for old versions of Python

In [ ]:
cat deadsnakes.list

deb http://ppa.launchpad.net/deadsnakes/ppa/ubuntu jammy main
deb-src http://ppa.launchpad.net/deadsnakes/ppa/ubuntu jammy main


## Install required Debian packages

### NVIDIA CUDA

In [ ]:
sudo apt -qq install cuda=12.0.1-1 cuda-drivers=525.85.12-1 cuda-11-7 cuda-11-8

cuda is already the newest version (12.0.1-1).
cuda-drivers is already the newest version (525.85.12-1).
cuda-11-7 is already the newest version (11.7.1-1).
cuda-11-8 is already the newest version (11.8.0-1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


### NVIDIA Container Runtime

In [ ]:
sudo apt -qq install nvidia-container-runtime

nvidia-container-runtime is already the newest version (3.12.0-1).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


### NVIDIA cuDNN

TBA!

### NVIDIA TensorRT

TBA!

### Xformers

TBA!

## Install Rust

Some Python modules now depend on Rust to build. Also, I wanted to build Anki from source. I decided to install Rust system-wide, in `/opt/rust`:

In [ ]:
cd /tmp
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs > install_rust.sh
sudo RUSTUP_HOME=/opt/rust CARGO_HOME=/opt/rust sh ./install_rust.sh -y --no-modify-path

In [ ]:
. /opt/rust/env
rustup default stable

## Python environments

I'm going to add Python venvs under `/opt/venvs`, and use hard linking to share large files between them.

| Folder | Depends | Uses | Purpose |
| --- | --- | --- | --- |
| python3.10-ai | Debian's python3.10 | torch stable | AI development |
| python3.10-webui | python3.10-ai | torch stable | stable-diffusion-webui |

In [ ]:
mkdir -p /opt/venvs
cd /opt/venvs

### The base venv python3.10-ai

In [ ]:
mkdir -p python3.10-ai
python3.10 -m venv python3.10-ai/venv

In [ ]:
lecho torch pipdeptree torchvision torchaudio tensorflow jupyter jupyterlab \
    ipywidgets bash_kernel jupyter-c-kernel nbdev pandas matplotlib \
    scipy scikit-learn scikit-image gradio onnx \
    transformers diffusers huggingface_hub numba \
    accelerate timm safetensors fastai fastbook \
    > python3.10-ai/require.txt

In [ ]:
(
. python3.10-ai/venv/bin/activate &&
pip install -qq -U -r python3.10-ai/require.txt
python -m bash_kernel.install
install_c_kernel
pip freeze | sort > python3.10-ai/freeze.txt
)

Installing IPython kernel spec
Installing IPython kernel spec
/opt/venvs/python3.10-ai/venv/bin/install_c_kernel:32: DeprecationWarning: replace is ignored. Installing a kernelspec always replaces an existing installation
  KernelSpecManager().install_kernel_spec(td, 'c', user=user, replace=True, prefix=prefix)


In [ ]:
cat <<END > python3.10-ai/venv/bin/pydoc
#!/bin/sh
python -m pydoc "$@"
END

chmod +x python3.10-ai/venv/bin/pydoc

### The secondary venv python3.10-webui

In [ ]:
mkdir -p python3.10-webui

In [ ]:
command rm -rf python3.10-webui/venv

In [ ]:
cp -al python3.10-ai/venv python3.10-webui/venv

In [ ]:
yes | venv_move python3.10-webui/venv

In [ ]:
(
cat python3.10-ai/require.txt ~/soft-ai/stable-diffusion-webui/requirements.txt 
echo fastapi==0.90.1
) > python3.10-webui/require.txt

In [ ]:
(
. python3.10-webui/venv/bin/activate &&
pip install -qq -U -r python3.10-webui/require.txt
pip freeze | sort > python3.10-webui/freeze.txt
)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
open-clip-torch 2.7.0 requires protobuf==3.20.0, but you have protobuf 3.19.6 which is incompatible.



In [ ]:
ln -f python3.10-ai/venv/bin/pydoc python3.10-webui/venv/bin/pydoc

### Compare the two venvs

In [ ]:
comm -3 python3.10-ai/freeze.txt python3.10-webui/freeze.txt | tee comm.txt

In [ ]:
git init
echo venv > .gitignore
git add -A
git commit -m 'venvs'

The `du` tool shows the size of the base venv, and the additional storage used by the webui venv. It only counts hard-linked files once. I'm saving nearly 6GB by using hard links:

In [ ]:
du -csh ./python3.10-{ai,webui}

5.9G	./python3.10-ai
794M	./python3.10-webui
6.6G	total


## NVIDIA tensorrt doesn't like python3.11

In [ ]:
sudo apt-get -q install python3-libnvinfer-dev

Reading package lists...
Building dependency tree...
Reading state information...
Some packages could not be installed. This may mean that you have
requested an impossible situation or if you are using the unstable
distribution that some required packages have not yet been created
or been moved out of Incoming.
The following information may help to resolve the situation:

The following packages have unmet dependencies:
 python3-libnvinfer : Depends: python3 (< 3.11) but 3.11.2-1 is to be installed
E: Unable to correct problems, you have held broken packages.


: 100

I'd like to install it as a python3.10 library at least, but it is demanding that the system Python version, i.e. the version of Debian's python3 package, should be less than 3.11, and I don't want to try to change that. So I'll adjust this `python3-libnvinfer` package to depend on the `python3.10` package instead of `python3 < 3.11`.

In [ ]:
cd /var/nv-tensorrt-local-repo-ubuntu2204-8.5.3-cuda-11.8
ls python3-libnvinfer*

python3-libnvinfer_8.5.3-1+cuda11.8_amd64.deb
python3-libnvinfer-dev_8.5.3-1+cuda11.8_amd64.deb


In [ ]:
cp -v python3-libnvinfer* ~/soft-ai/

'python3-libnvinfer_8.5.3-1+cuda11.8_amd64.deb' -> '/home/sam/soft-ai/python3-libnvinfer_8.5.3-1+cuda11.8_amd64.deb'
'python3-libnvinfer-dev_8.5.3-1+cuda11.8_amd64.deb' -> '/home/sam/soft-ai/python3-libnvinfer-dev_8.5.3-1+cuda11.8_amd64.deb'


In [ ]:
cd ~/soft-ai
command rm -rf unpacked
dpkg-deb -R python3-libnvinfer_8.5.3-1+cuda11.8_amd64.deb unpacked

In [ ]:
grep Depends unpacked/DEBIAN/control

Depends: python3 (>= 3.10), python3 (<< 3.11), libnvinfer8 (= 8.5.3-1+cuda11.8), libnvinfer-plugin8 (= 8.5.3-1+cuda11.8), libnvparsers8 (= 8.5.3-1+cuda11.8), libnvonnxparsers8 (= 8.5.3-1+cuda11.8)


In [ ]:
sed -i 's/python3 (>= 3.10), python3 (<< 3.11), /python3.10, /' unpacked/DEBIAN/control
grep Depends unpacked/DEBIAN/control

Depends: python3.10, libnvinfer8 (= 8.5.3-1+cuda11.8), libnvinfer-plugin8 (= 8.5.3-1+cuda11.8), libnvparsers8 (= 8.5.3-1+cuda11.8), libnvonnxparsers8 (= 8.5.3-1+cuda11.8)


In [ ]:
dpkg-deb -b unpacked python3-libnvinfer_8.5.3-1+cuda11.8_amd64_fixed.deb

dpkg-deb: building package 'python3-libnvinfer' in 'python3-libnvinfer_8.5.3-1+cuda11.8_amd64_fixed.deb'.


In [ ]:
sudo dpkg -i ./python3-libnvinfer_8.5.3-1+cuda11.8_amd64_fixed.deb ./python3-libnvinfer-dev_8.5.3-1+cuda11.8_amd64.deb

Selecting previously unselected package python3-libnvinfer.
(Reading database ... 1157567 files and directories currently installed.)
Preparing to unpack .../python3-libnvinfer_8.5.3-1+cuda11.8_amd64_fixed.deb ...
Unpacking python3-libnvinfer (8.5.3-1+cuda11.8) ...
Selecting previously unselected package python3-libnvinfer-dev.
Preparing to unpack .../python3-libnvinfer-dev_8.5.3-1+cuda11.8_amd64.deb ...
Unpacking python3-libnvinfer-dev (8.5.3-1+cuda11.8) ...
Setting up python3-libnvinfer (8.5.3-1+cuda11.8) ...
Setting up python3-libnvinfer-dev (8.5.3-1+cuda11.8) ...


In [ ]:
# __END__

# Graveyard of Bad Ideas

Here is a list of other things I tried, which didn't work out so well.

## Allow myself to install packages to /usr/local

Since I wrote this section, Debian has modified their pip so that it refuses to install to `/usr/local`, unless we use the `--break-system-packages` flag.


It's not safe to run `pip install -U` as root. It will merrily remove files from Debian-packaged Python modules under `/usr/lib`, and mess up the system. To avoid this, I changed permissions so that my regular user account can write to everything under `/usr/local` via the `staff` group. I do the same for `/opt` because I want to upgrade Rust from my own user ID also:

In [ ]:
sudo adduser $USER staff
sudo chgrp -R staff /usr/local /opt
sudo chmod -R g+w /usr/local /opt

This is somewhat of a security concern, but if a bad guy gets a local shell it's pretty much game over anyway. To make it safer, I could have used a different non-root account.

In order to upgrade a pip package where the same package was also installed in `/usr/lib` by dpkg, I need to use `pip -I -U packagename`. The `-I` flag tells it to ignore installed packages, i.e. don't try to remove them. I only do that as needed for individual packages, when a normal upgrade fails.

This setup might upset Debian-packaged programs that depend on older versions of the Python libraries that I've upgraded. I haven't noticed any problems like that yet.

If you try to downgrade a package, and if multiple versions of a package are accidentally installed under /usr/local/lib/python3.10 at that same time, it can get confused, so watch out for that. You might need to manually remove one of the versions.

## Enable switching the system python version

The default Python version for Debian "testing" at time of writing is Python 3.11, however we don't have a stable release of pytorch for Python 3.11 yet. I decided to try switch the system Python version back to 3.10. This is not recommended, but at least it's easy to undo!

In [ ]:
python3.11 -m pip install --break-system-packages torchvision

ERROR: Could not find a version that satisfies the requirement torchvision (from versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3)
ERROR: No matching distribution found for torchvision



: 1

Anyway, let's enable switching the system Python version back to 3.10, using `update-alternatives`.

First, note that we can undo this change later, if necessary, as follows:

In [ ]:
sudo update-alternatives --remove-all python3
sudo update-alternatives --remove-all pydoc3
sudo ln -sf python3.11 /usr/bin/python3
sudo ln -sf pydoc3.11 /usr/bin/pydoc3

We can set up alternatives for `python3`:

In [ ]:
sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 11
sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 10
# sudo update-alternatives --set python3 /usr/bin/python3.10

and do the same for `pydoc3`:

In [ ]:
sudo update-alternatives --install /usr/bin/pydoc3 pydoc3 /usr/bin/pydoc3.11 11
sudo update-alternatives --install /usr/bin/pydoc3 pydoc3 /usr/bin/pydoc3.10 10
# sudo update-alternatives --set pydoc3 /usr/bin/pydoc3.10

In [ ]:
update-alternatives --list python3

In [ ]:
update-alternatives --list pydoc3

## Install nightly torch with Python 3.11

In [ ]:
pip install -qq -U fastbook jupyter jupyterlab tensorflow ipywidgets

At some point I saw a warning about compatibility between fastai and torch.
```
fastai 2.7.11 has requirement torch<1.14,>=1.7, but you have torch 2.0.0.dev20230220+cu117
```
I decided to try using it with nightly torch anyway:

In [ ]:
pip install -qq -U --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu117

Extra packages: nbdev, and kernels for bash and C:

In [ ]:
pip install -qq -U nbdev bash_kernel jupyter-c-kernel

## Use the old pip resolver, if necessary

As of pip 20.3, a new resolver has been introduced, which doesn't always work. As of pip 21.0 the old working resolver is unsupported and slated for removal dependent on pip team resources.

In [ ]:
pip --version

pip 23.0 from /usr/lib/python3/dist-packages/pip (python 3.11)


With this version of pip, it's possible to use the "legacy resolver" with the following option:

In [ ]:
pip install -U -r requirements.txt --use-deprecated=legacy-resolver

Or, I can downgrade pip to version 20.2.4, before they switched to the new resolver.

Refer to https://stackoverflow.com/a/67408694 for more info.

In [ ]:
pip install pip==20.2.4

## PyPI does not include onnxruntime for python3.11

At one point I was hoping to use python3.11. I attempted to build onnxruntime from source, but didn't persevere with it.

## Untold difficulties

I haven't described all of the various problems I had to deal with, due to using various bad approaches rather than venvs. There were plenty of problems! I guess I learned something from it, anyway. I wish Lambda Stack worked on Debian, not only Ubuntu, it would have been a lot easier than this. Maybe I should give it a try on Debian anyway, and see what happens.